# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

## NetCDF configuration package and visualization
  - Additional NetCDF configuration options are available in the `NCF6` package
  - These include a `WKT` (well-known text) string that can aid visualization in applications like QGIS or Panoply

In [ ]:
import pathlib as pl

import flopy
import matplotlib.pyplot as plt
import xarray as xa
import xugrid as xu

### Load the base watershed model
  - Use the base watershed model to demonstrate `MODFLOW 6` `NCF` package functionality. 

In [ ]:
name = "watershed"
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("data/hatari_regional/")
)

### Modify the model to save NetCDF output
  - Set `start_date_time` in the TDIS file
  - Define `export_netcdf` in the groundwater flow model `name_file`, specifying `ugrid` as the output type
  - Activate `DIS` package array export with the `EXPORT_ARRAY_NETCDF` keyword in the package `OPTION` block 

In [ ]:
sim.tdis.start_date_time = "1980-01-01"

In [ ]:
gwf = sim.get_model()
gwf.export_netcdf = "ugrid"
gwf.disv.export_array_netcdf = True

### Define `WKT` string and create `NCF` package
  - The `NCF6` package is currently a subpackage of the model discretization (`DIS` or `DISV`) package
  - Currently this package contains options related to visualization and compression

In [ ]:
wkt = """PROJCRS["WGS 84 / UTM zone 18S",BASEGEOGCRS["WGS 84",ENSEMBLE["World Geodetic System 1984 ensemble",MEMBER["World Geodetic System 1984 (Transit)"],MEMBER["World Geodetic System 1984 (G730)"],MEMBER["World Geodetic System 1984 (G873)"],MEMBER["World Geodetic System 1984 (G1150)"],MEMBER["World Geodetic System 1984 (G1674)"],MEMBER["World Geodetic System 1984 (G1762)"],MEMBER["World Geodetic System 1984 (G2139)"],MEMBER["World Geodetic System 1984 (G2296)"],ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ENSEMBLEACCURACY[2.0]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["UTM zone 18S",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",-75,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",500000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",10000000,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["Navigation and medium accuracy spatial referencing."],AREA["Between 78°W and 72°W, southern hemisphere between 80°S and equator, onshore and offshore. Argentina. Brazil. Chile. Colombia. Ecuador. Peru."],BBOX[-80,-78,0,-72]],ID["EPSG",32718]]"""

ncf = flopy.mf6.ModflowUtlncf(
    gwf.disv,
    ogc_wkt=wkt,
    filename="regional_model.disv.ncf",
)

### Run the model
  - Change simulation workspace
  - Write model files and run the model

In [ ]:
ws = pl.Path("temp/step3")
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()